In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)

import warnings

warnings.filterwarnings(action="ignore")

In [ ]:
def preprocessing(df):
    df = df.copy()
    #     Binary Encoding of gender
    df["SEX"] = df["SEX"].replace({"F": 0, "M": 1})

    return df

def split_data(df):
    # Splitting DF into X and y
    y = df["SOURCE"]
    X = df.drop("SOURCE", axis=1)

    X_train, x_test, y_train, y_test = train_test_split(
        X, y, train_size=0.7, shuffle=True, random_state=1
    )

    #    Scaling through Standard Scaler
    sc = StandardScaler()
    sc.fit(X_train)

    X_train = pd.DataFrame(
        sc.transform(X_train), columns=X_train.columns, index=X_train.index
    )
    x_test = pd.DataFrame(
        sc.transform(x_test), columns=x_test.columns, index=x_test.index
    )

    return X_train, x_test, y_train, y_test

In [ ]:
df = pd.read_csv("../data/patient_treatment/data-ori.csv")

df.head()

In [ ]:
df = preprocessing(df)
X_train, x_test, y_train, y_test = split_data(df)

# First Approach

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(),
    "      Decision Tree": DecisionTreeClassifier(),
    "     Neural Network": MLPClassifier(),
    "      Random Forest": RandomForestClassifier(),
    "  Gradient Boosting": GradientBoostingClassifier(),
}

for k, v in models.items():
    v.fit(X_train, y_train)
    print(k + " Trained !")

In [ ]:
for k, v in models.items():
    y_pred = v.predict(x_test)
    acc = accuracy_score(y_test, y_pred)
    print(k + " Accuracy : {:.2f}%".format(acc * 100))

In [ ]:
for k, v in models.items():
    y_pred = v.predict(x_test)
    f1 = f1_score(y_test, y_pred, pos_label="in")
    print(k + " F1 score : {:.5f}".format(f1))

# Second Approach

In [ ]:
import shap

In [ ]:
# initialize data
y = df[main_label].values.reshape(
    -1,
)
X = df.drop([main_label], axis=1)
cat_cols = df.select_dtypes(include=["object"]).columns
cat_cols_idx = [list(X.columns).index(c) for c in cat_cols]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape



In [ ]:
# initialize Pool
train_pool = Pool(X_train, y_train, cat_features=cat_cols_idx)
test_pool = Pool(X_test, y_test, cat_features=cat_cols_idx)
# specify the training parameters
model = CatBoostClassifier(
    iterations=1000,
    depth=5,
    border_count=23,
    l2_leaf_reg=0.3,
    learning_rate=3e-3,
    verbose=0,
)

# train the model
model.fit(train_pool)
# make the prediction using the resulting model
y_train_pred = model.predict_proba(train_pool)[:, 1]
y_test_pred = model.predict_proba(test_pool)[:, 1]
roc_auc_train = roc_auc_score(y_train, y_train_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)
print(
    f"ROC AUC score for train {round(roc_auc_train,4)}, and for test {round(roc_auc_test,4)}"
)

In [ ]:
# calculating the baseline ROC AUC score assuming the same probability from training labels to test
roc_auc_baseline = roc_auc_score(y_test, [np.mean(y_train)] * len(y_test))
print(roc_auc_baseline)

## Evaluation

### Shapley values
- Feature contribution to model's prediction
- Interpretability: Providing clear insights to model behavior
- Global & Local Explanations: They explain both overall feature importance (global) and individual predictions (local).
- Model-Agnostic & Model-Specific Methods: SHAP can be applied to tree-based models, neural networks, and other ML algorithms.

SHAP values come from Shapley values, which originate from cooperative game theory. The idea is to fairly distribute the total "payout" (model prediction) among the "players" (features) based on their contributions.

### SHAP Value Formula

For a given feature \( i \), its SHAP value is calculated as:

$
\phi_i = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|! (|N| - |S| - 1)!}{|N|!} \left[ f(S \cup \{i\}) - f(S) \right]
$

where:
- \( N \) is the set of all features.
- \( S \) is a subset of features excluding \( i \).
- \( f(S) \) is the model’s prediction when using only the features in \( S \).
- \( f(S \cup \{i\}) \) is the model’s prediction when adding feature \( i \) to subset \( S \).
- The fraction is a weighting term ensuring fairness by averaging contributions over all possible subsets.

### Example SHAP Calculation for a Feature (e.g., Size)

For each subset \( S \), we compute:

$\Delta f = f(S \cup \{Size\}) - f(S)$

The SHAP value for "Size" is then:

$\phi_{Size} = \frac{1}{3} (50K) + \frac{1}{3} (50K) + \frac{1}{6} (60K) + \frac{1}{6} (60K)

= 16.67K + 16.67K + 10K + 10K = 53.3K
$

Thus, the SHAP value for **Size** is **53.3K**, meaning "Size" contributes 53.3K to the final prediction.


Cons: 
- The computation involves testing every possible combination of features, making it computationally expensive for large models

In [ ]:
shap.initjs()
ex = shap.TreeExplainer(model)
print(f"Average treatment probability is {round(np.mean(y_test),4)}")
shap_values = ex.shap_values(X_test)
shap.summary_plot(shap_values, X_test, max_display=30)